In [3]:
# Setup and Import Libraries
import pandas as pd
from sqlalchemy import create_engine

In [4]:
# Database Connection
username = username
password = password
host = "postgres.host.amazonaws.com"
port = "5432"
database = "postgres"

# Create the database URL
db_url = f"postgresql://{username}:{password}@{host}:{port}/{database}"

engine = create_engine(db_url)

In [5]:
db_url = 'postgresql://password@postgres.c3ec2o0eqflu.us-east-2.rds.amazonaws.com:5432/postgres'

engine = create_engine(db_url)

A SQL query is executed to fetch data from the database. The query groups ads by user and category, counts the number of ads per category, and assigns a rank based on the number of ads within each user group:

In [6]:
# SQL Query Execution
sql_query = '''
SELECT A.USER_ID,
             A.CATEGORY_ID,
             A.CATEGORY_NAME,
             COUNT(A.CATEGORY_ID)                       AS ADS,
             ROW_NUMBER() OVER (PARTITION BY A.USER_ID) AS CATEGORY_RANK
      FROM ADS A
      GROUP BY A.USER_ID, A.CATEGORY_ID, A.CATEGORY_NAME
      ORDER BY 1, 5;
'''

# Loading Data into DataFrame
df = pd.read_sql(sql_query, engine)
df.head()

,user_id,category_id,category_name,ads,category_rank
0,449,37,electronic,1,1
1,10327,1,real estate,1,1
2,20342,3,car parts,1,1
3,21747,891,fashion and style,3,1
4,23066,891,fashion and style,1,1


In [47]:
# Data Transformation and Analysis
pivot_table = df.pivot_table(index='user_id', columns='category_name', aggfunc='size', fill_value=0)

# Calculate the overlap matrix
overlap_df = pivot_table.T.dot(pivot_table)

overlap_df.head(30)

category_name,animals,business and services,car parts,children,electronic,fashion and style,"hobby, rest and sport",home and garden,jobs,real estate,transport
category_name,,,,,,,,,,,
animals,23865,383,388,1022,1051,1427,799,1181,112,154,241
business and services,383,37285,1000,1178,2131,2000,1130,2342,796,367,566
car parts,388,1000,55184,1477,4424,2101,2017,3075,197,235,1920
children,1022,1178,1477,104731,6059,17466,5723,5725,336,512,457
electronic,1051,2131,4424,6059,146533,10283,6885,8488,637,689,1245
fashion and style,1427,2000,2101,17466,10283,150964,8518,8542,715,756,600
"hobby, rest and sport",799,1130,2017,5723,6885,8518,61327,6041,261,330,529
home and garden,1181,2342,3075,5725,8488,8542,6041,77636,401,662,804
jobs,112,796,197,336,637,715,261,401,17257,174,165


In [36]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
from googleapiclient.discovery import build

In [21]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

In [26]:
creds =ServiceAccountCredentials.from_json_keyfile_name('/Users/danielpavelko/Downloads/olx-daniel-pavelko-1892afec055d.json', scope)

In [27]:
# Authorize the clientsheet 
client = gspread.authorize(creds)

In [28]:
# Create the spreadsheet
spreadsheet_name = 'OLX Pavelko Daniel - XX:XXXX:XX'

spreadsheet = client.create(spreadsheet_name)

print("Spreadsheet '{spreadsheet_name}' created successfully.")

Spreadsheet 'OLX Pavelko Daniel - 2024-07-11' created successfully.


In [29]:
# Select the first sheet in the newly created spreadsheet
worksheet = spreadsheet.get_worksheet(0)

# Rename the first sheet to "Test task"
worksheet.update_title("Test task")

# Convert the DataFrame to a list of lists for updating the Google Sheet
data_to_upload = [overlap_df.columns.values.tolist()] + overlap_df.reset_index().values.tolist()

# Update the worksheet with the data
worksheet.update(data_to_upload)

print(f"Spreadsheet '{spreadsheet_name}' with sheet 'Test task' updated successfully.")

Spreadsheet 'OLX Pavelko Daniel - 2024-07-11' with sheet 'Test task' updated successfully.


In [32]:
spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}/edit"
print(f"Open the spreadsheet here: {spreadsheet_url}")

Open the spreadsheet here: https://docs.google.com/spreadsheets/d/192Nm8ARcdJq0XhigBN7F6jSb9zl-clm6h9kr4oOvyho/edit


In [48]:
drive_service = build('drive', 'v3', credentials=creds)

permission = {
    'type': 'anyone',
    'role': 'reader'
}
drive_service.permissions().create(
    fileId=spreadsheet.id,
    body=permission,
    fields='id'
).execute()

{'id': 'anyoneWithLink'}